In [1]:
import pandas as pd
from os import getcwd
%load_ext autoreload
%autoreload 2


raw_data_dir = f"{getcwd()}/../data/raw"
processed_data_dir = f"{getcwd()}/../data/processed"

In [2]:
filtered_df = pd.read_csv(f"{processed_data_dir}/filtered_data.csv")


In [3]:
filtered_df.head(100)

,cik,Name,Year,IncomeTaxReconciliationStateAndLocalIncomeTaxes,ExcessTaxBenefitFromShareBasedCompensationFinancingActivities,NetIncomeLossAttributableToNoncontrollingInterest,ResearchAndDevelopmentExpense,ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsGrantsInPeriodWeightedAverageGrantDateFairValue,ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsOutstandingIntrinsicValue,IncomeTaxExpenseBenefit,...,EmployeeServiceShareBasedCompensationTaxBenefitFromCompensationExpense,FiniteLivedIntangibleAssetsAmortizationExpenseYearTwo,ComprehensiveIncomeNetOfTaxIncludingPortionAttributableToNoncontrollingInterest,NetIncomeLossAvailableToCommonStockholdersBasic,IncreaseDecreaseInAccountsPayableAndAccruedLiabilities,InterestExpense,RetainedEarningsAccumulatedDeficit,InterestIncomeExpenseNet,IntangibleAssetsNetExcludingGoodwill,eps
0,1141807,1ST CONSTITUTION BANCORP,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3212
1,1141807,1ST CONSTITUTION BANCORP,2010,NaN,NaN,NaN,NaN,NaN,NaN,1.267038e+06,...,NaN,NaN,NaN,2286053.0,NaN,8819498.0,1.074178e+07,20481521.0,NaN,0.4689
2,1141807,1ST CONSTITUTION BANCORP,2011,86913.0,NaN,NaN,NaN,NaN,16122.0,1.291131e+06,...,NaN,NaN,NaN,3931443.0,NaN,6787364.0,1.307061e+07,23069682.0,NaN,0.6652
3,1141807,1ST CONSTITUTION BANCORP,2012,196959.0,NaN,NaN,NaN,NaN,16122.0,4.386420e+05,...,NaN,NaN,NaN,NaN,NaN,397920.0,1.559429e+07,27685494.0,NaN,0.8163
4,1141807,1ST CONSTITUTION BANCORP,2013,280000.0,NaN,NaN,NaN,NaN,623916.0,2.285000e+06,...,NaN,231255.0,NaN,NaN,NaN,1012000.0,2.137400e+07,24737000.0,NaN,0.8798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1670541,ADIENT PLC,2016,0.0,NaN,84000000.0,460000000.0,13.15,NaN,1.839000e+09,...,11000000.0,NaN,-1.496000e+09,NaN,57000000.0,NaN,0.000000e+00,-22000000.0,113000000.0,9.3500
96,1670541,ADIENT PLC,2017,NaN,NaN,85000000.0,488000000.0,NaN,NaN,9.900000e+07,...,21000000.0,49000000.0,8.460000e+08,NaN,-113000000.0,NaN,7.340000e+08,-132000000.0,543000000.0,5.6200
97,1670541,ADIENT PLC,2018,NaN,NaN,84000000.0,513000000.0,NaN,6000000.0,4.800000e+08,...,0.0,39000000.0,-1.727000e+09,NaN,143000000.0,NaN,-1.028000e+09,-144000000.0,460000000.0,1.6300
98,1670541,ADIENT PLC,2019,-41000000.0,NaN,83000000.0,454000000.0,NaN,4000000.0,4.100000e+08,...,0.0,37000000.0,-4.460000e+08,NaN,-191000000.0,NaN,-1.545000e+09,-182000000.0,405000000.0,-0.0400


In [6]:
def change(a, b):
    if a == 0:
        return 0
    return (b - a) / abs(a)

non_change_cols = {'cik', 'Name', "Year"}

def get_change(row):
    last_year_row = filtered_df[(filtered_df["Year"] == row["Year"] - 1) & (filtered_df['cik'] == row['cik'])]
    if last_year_row.shape[0] == 0:
        return None
    last_year_row = last_year_row.iloc[0]

    for col in row.index:
        if col in non_change_cols:
            continue
        if pd.isna(row[col]) or pd.isna(last_year_row[col]):
            row.loc[col] = 0
        elif col not in non_change_cols:
            row.loc[col] = change(last_year_row[col], row[col])
    return row



In [92]:
new_df = filtered_df.apply(get_change, axis=1)

In [93]:
filtered_df["Year"].head()

0    2009
1    2010
2    2011
3    2012
4    2013
Name: Year, dtype: int64

In [94]:
new_df.head()

0                                                 None
1    cik                                           ...
2    cik                                           ...
3    cik                                           ...
4    cik                                           ...
dtype: object

In [95]:
new_df.dropna()

1       cik                                           ...
2       cik                                           ...
3       cik                                           ...
4       cik                                           ...
6       cik                                           ...
                              ...                        
5777    cik                                           ...
5778    cik                                           ...
5779    cik                                           ...
5780    cik                                           ...
5781    cik                                           ...
Length: 5036, dtype: object

In [96]:
new_df.iloc[2]

cik                                                                               1141807
Name                                                             1ST CONSTITUTION BANCORP
Year                                                                                 2011
IncomeTaxReconciliationStateAndLocalIncomeTaxes                                         0
ExcessTaxBenefitFromShareBasedCompensationFinancingActivities                           0
                                                                           ...           
InterestExpense                                                                 -0.230414
RetainedEarningsAccumulatedDeficit                                               0.216801
InterestIncomeExpenseNet                                                         0.126366
IntangibleAssetsNetExcludingGoodwill                                                    0
eps                                                                              0.418639
Name: 2, L

In [97]:
new_df.iloc[5]

In [98]:
new_df.iloc[6]

cik                                                                               1141807
Name                                                             1ST CONSTITUTION BANCORP
Year                                                                                 2016
IncomeTaxReconciliationStateAndLocalIncomeTaxes                                  0.163121
ExcessTaxBenefitFromShareBasedCompensationFinancingActivities                           0
                                                                           ...           
InterestExpense                                                                  0.163248
RetainedEarningsAccumulatedDeficit                                               0.331588
InterestIncomeExpenseNet                                                         0.039171
IntangibleAssetsNetExcludingGoodwill                                                    0
eps                                                                              0.065421
Name: 6, L

In [126]:
li = []
for index in new_df.index:
    if new_df.iloc[index] is not None:
        li.append(new_df.iloc[index].to_frame().transpose())
li

[       cik                      Name  Year  \
 1  1141807  1ST CONSTITUTION BANCORP  2010   
 
   IncomeTaxReconciliationStateAndLocalIncomeTaxes  \
 1                                               0   
 
   ExcessTaxBenefitFromShareBasedCompensationFinancingActivities  \
 1                                                  0              
 
   NetIncomeLossAttributableToNoncontrollingInterest  \
 1                                                 0   
 
   ResearchAndDevelopmentExpense  \
 1                             0   
 
   ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsGrantsInPeriodWeightedAverageGrantDateFairValue  \
 1                                                  0                                                                
 
   ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsOutstandingIntrinsicValue  \
 1                                                  0                                          
 
   IncomeTaxExpenseBenefit  ...  \
 

In [127]:
new_df_df = pd.concat(li)

In [128]:
new_df_df

,cik,Name,Year,IncomeTaxReconciliationStateAndLocalIncomeTaxes,ExcessTaxBenefitFromShareBasedCompensationFinancingActivities,NetIncomeLossAttributableToNoncontrollingInterest,ResearchAndDevelopmentExpense,ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsGrantsInPeriodWeightedAverageGrantDateFairValue,ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsOutstandingIntrinsicValue,IncomeTaxExpenseBenefit,...,EmployeeServiceShareBasedCompensationTaxBenefitFromCompensationExpense,FiniteLivedIntangibleAssetsAmortizationExpenseYearTwo,ComprehensiveIncomeNetOfTaxIncludingPortionAttributableToNoncontrollingInterest,NetIncomeLossAvailableToCommonStockholdersBasic,IncreaseDecreaseInAccountsPayableAndAccruedLiabilities,InterestExpense,RetainedEarningsAccumulatedDeficit,InterestIncomeExpenseNet,IntangibleAssetsNetExcludingGoodwill,eps
1,1141807,1ST CONSTITUTION BANCORP,2010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.459838
2,1141807,1ST CONSTITUTION BANCORP,2011,0,0,0,0,0,0,0.019015,...,0,0,0,0.719751,0,-0.230414,0.216801,0.126366,0,0.418639
3,1141807,1ST CONSTITUTION BANCORP,2012,1.266163,0,0,0,0,0.0,-0.660265,...,0,0,0,0,0,-0.941373,0.193081,0.200081,0,0.22715
4,1141807,1ST CONSTITUTION BANCORP,2013,0.421616,0,0,0,0,37.699665,4.209259,...,0,0,0,0,0,1.543225,0.37063,-0.1065,0,0.07779
6,1141807,1ST CONSTITUTION BANCORP,2016,0.163121,0,0,0,0,1.18583,5.188755,...,0,-0.205729,0,0,0,0.163248,0.331588,0.039171,0,0.065421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5777,855612,ZIX CORP,2016,0.682171,0,0,0.148611,0,0,0.1743,...,0,0,0,0,0,0,0.024928,0,0,0.238095
5778,855612,ZIX CORP,2017,-1.59447,0,0,0.149377,0.364238,0,4.039545,...,0,0,0,0,0,0,-0.035289,0,0,0.115385
5779,855612,ZIX CORP,2018,7.317829,0,0,0.031239,-1.0,0,-1.253682,...,0,1.752443,0,2.916842,0,0,0.084646,0,1.760862,0.137931
5780,855612,ZIX CORP,2019,-1.622086,0,0,0.80438,0,0,0.051271,...,0,14.092308,0,-2.594859,0,0,-0.113841,0,8.565012,0.30303


In [129]:
type(new_df)

pandas.core.series.Series

In [110]:
new_df_df = new_df.to_frame().transpose()

In [132]:
new_df_df

In [7]:
new_df_df = new_df_df.to_csv(f"{processed_data_dir}/change_data.csv", index=False)

NameError: name 'new_df_df' is not defined

In [2]:
new_df_df = pd.read_csv(f"{processed_data_dir}/change_data.csv")

In [3]:
new_df_df['eps'] = new_df_df['eps'].apply(lambda x: 1 if x >= 0 else 0)

In [ ]:
new_df_df = new_df_df.to_csv(f"{processed_data_dir}/change_data.csv", index=False)